In [1]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

ModuleNotFoundError: No module named 'object_detection'

In [4]:
!cp -R /home/tensorflow/models/research/object_detection .

cp: cannot access '/home/tensorflow/models/research/object_detection/test_data/checkpoint': Permission denied
